In [28]:
import os
from dotenv import load_dotenv 
load_dotenv() 
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [29]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper 

### **Wikipedia tool**

In [30]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

### **Chunking and splitting**

In [31]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS 
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://medium.com/@zilliz_learn/graphrag-explained-enhancing-rag-with-knowledge-graphs-3312065f99e1")
docs = loader.load()

In [32]:
text_split = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200)
documents = text_split.split_documents(docs)

In [33]:
db = FAISS.from_documents(documents, OpenAIEmbeddings())
retriever = db.as_retriever()

### **Retrival Tool**

In [34]:
from langchain.tools.retriever import create_retriever_tool 
retriever_tool = create_retriever_tool(retriever, "GraphRAG", "Search for information about GraphRAG. For any question about GraphRAG, you must use this tool!")

In [35]:
retriever_tool.name

'GraphRAG'

### **Arxiv tool**

In [36]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max = 200)
arxiv = ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [37]:
tools = [wiki, arxiv, retriever_tool]

In [38]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'd:\\Code\\AgenticAI\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='GraphRAG', description='Search for information about GraphRAG. For any question about GraphRAG, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001C7DF25BB00>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.

In [39]:
from langchain import hub

llm = ChatOpenAI(model = "gpt-4-turbo")
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [40]:
### Agents 

from langchain.agents import create_openai_tools_agent, create_openai_functions_agent
agent = create_openai_tools_agent(llm, tools, prompt)

In [41]:
### Agent Executer
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag

In [42]:
response = agent_executor.invoke({"input": "Tell about GraphRAG"})



> Entering new AgentExecutor chain...

Invoking: `GraphRAG` with `{'query': 'GraphRAG'}`


combined with the Milvus vector database, GraphRAG can navigate intricate semantic relationships within large datasets, providing more accurate and insightful results. This powerful combination makes GraphRAG an invaluable asset for various practical GenAI applications, providing a robust solution for understanding and processing complex information.Further ResourcesThe GraphRAG Paper: From Local to Global: A Graph RAG Approach to Query-Focused SummarizationGraphRAG GitHub: https://github.com/microsoft/graphragOther RAG enhancement techniques:Better RAG with HyDE — Hypothetical Document EmbeddingsEnhancing RAG with Knowledge Graphs using WhyHowHow to Enhance the Performance of Your RAG PipelineOptimizing RAG with Rerankers: The Role and Trade-offsPractical Tips and Tricks for Developers Building RAG ApplicationsInfrastructure Challenges in Scaling RAG with Custom AI Models----3FollowWritten by 

In [43]:
agent_executor.invoke({"input": "What is are AI Agents"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'AI Agents'}`


Page: Intelligent agent
Summary: In intelligence and artificial intelligence, an intelligent agent (IA) is an agent that perceives its environment, takes actions autonomously in order to achieve goalsAn AI (Artificial Intelligence) agent, also known as an intelligent agent (IA), is an entity that perceives its environment and takes autonomous actions to achieve specific goals. These agents use AI techniques to analyze inputs from their surroundings and make decisions that maximize their chances of success in achieving their objectives. AI agents are used in various applications, ranging from virtual personal assistants to autonomous vehicles and sophisticated decision-making systems in business and science.

> Finished chain.


{'input': 'What is are AI Agents',
 'output': 'An AI (Artificial Intelligence) agent, also known as an intelligent agent (IA), is an entity that perceives its environment and takes autonomous actions to achieve specific goals. These agents use AI techniques to analyze inputs from their surroundings and make decisions that maximize their chances of success in achieving their objectives. AI agents are used in various applications, ranging from virtual personal assistants to autonomous vehicles and sophisticated decision-making systems in business and science.'}

In [44]:
agent_executor.invoke({"input": "Whats the paper 1605.08386 about ?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with the identifier "1605.08386" is titled "Heat-bath random walks with Markov bases." It was published on May 26, 2016, by authors Caprice Stanley and Tobias Windisch. The paper investigates graphs on lattice points where the edges are derived from a finite set of vectors, focusing on how these structures facilitate the exploration of specific mathematical or physical concepts, though the summary provided does not specify further details.

> Finished chain.


{'input': 'Whats the paper 1605.08386 about ?',
 'output': 'The paper with the identifier "1605.08386" is titled "Heat-bath random walks with Markov bases." It was published on May 26, 2016, by authors Caprice Stanley and Tobias Windisch. The paper investigates graphs on lattice points where the edges are derived from a finite set of vectors, focusing on how these structures facilitate the exploration of specific mathematical or physical concepts, though the summary provided does not specify further details.'}